<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/clics3Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

Create the dataframe from csv file

In [2]:
df = pd.read_csv('df_all_raw.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
2,abrahammonpa,BugunNamphri-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
3,abrahammonpa,BugunSingchung-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
4,abrahammonpa,BugunWangho-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0


Let's create a new dataframe with only relevant columns

In [4]:
# keep only relevant columns --> new df
df_concept = df[['Concepticon_ID','Concepticon_Gloss']].copy()

In [5]:
df_concept['Concepticon_ID'].isnull().values.any()

False

In [6]:
df_concept['Concepticon_Gloss'].isnull().values.any()

False

The dataframe doesn't contains null values

In [7]:
df_concept.info

<bound method DataFrame.info of          Concepticon_ID Concepticon_Gloss
0                  1369              GOLD
1                  1369              GOLD
2                  1369              GOLD
3                  1369              GOLD
4                  1369              GOLD
...                 ...               ...
1390589            1366               BOY
1390590            1366               BOY
1390591            1366               BOY
1390592            1366               BOY
1390593            1366               BOY

[1390594 rows x 2 columns]>

We can see that there are many duplicates. We want to remove them:

In [8]:
# remove duplicates
df_concept=df_concept.drop_duplicates()
# reset index
df_concept = df_concept.reset_index(drop=True)

In [9]:
df_concept.info

<bound method DataFrame.info of       Concepticon_ID     Concepticon_Gloss
0               1369                  GOLD
1               1035                  GOOD
2               1425                 GREEN
3               2648           HAIR (HEAD)
4               1481                HAMMER
...              ...                   ...
2914            2033               DELIVER
2915            3288                  WEED
2916            2529               BLESSED
2917            3305        INSIDE A HOUSE
2918            2473  HIS (GENITIVE OF HE)

[2919 rows x 2 columns]>

Let's remove multiword Concepticon rows

In [16]:
nWords = df_concept['Concepticon_Gloss'].str.count(' ') + 1
for i in len(nWords):
  if nWords(i) > 1:
    df_concept.drop([i])

TypeError: ignored

In [26]:
# create column with word count
df_concept['totalwords'] = df_concept['Concepticon_Gloss'].str.count(' ') + 1
# delete columns with more than one word
df_concept = df_concept.drop(df_concept[df_concept.totalwords > 1].index)
# delete column with word count
del df_concept['totalwords']

In [27]:
df_concept.head()

,Concepticon_ID,Concepticon_Gloss
0,1369,GOLD
1,1035,GOOD
2,1425,GREEN
4,1481,HAMMER
5,1277,HAND


For testing, we take only 10 rows

In [28]:
df_concept_small=df_concept[:10]

In [29]:
df_concept_small

,Concepticon_ID,Concepticon_Gloss
0,1369,GOLD
1,1035,GOOD
2,1425,GREEN
4,1481,HAMMER
5,1277,HAND
6,1256,HEAD
7,1223,HEART
8,1210,HEAVY
10,1741,ABOVE
11,1286,HOT


### Create df with combinations

NB: permutation when the order of selection is a factor, combination when order is not a factor

For example, if the original data frame has the following gloss/ID rows:
  * dog, 5
  * cat, 7
  * chair, 23
  
  The new one would have the following gloss_x / ID_x / gloss_y / ID_y rows:
  * dog, 5, cat, 7
  * dog, 5, chair, 23
  * cat, 7, chair 23

In [30]:
import itertools
# create list of combinations
comb=list(itertools.combinations(df_concept_small['Concepticon_Gloss'], 2))

In [31]:
print(comb)
print(len(comb))

[('GOLD', 'GOOD'), ('GOLD', 'GREEN'), ('GOLD', 'HAMMER'), ('GOLD', 'HAND'), ('GOLD', 'HEAD'), ('GOLD', 'HEART'), ('GOLD', 'HEAVY'), ('GOLD', 'ABOVE'), ('GOLD', 'HOT'), ('GOOD', 'GREEN'), ('GOOD', 'HAMMER'), ('GOOD', 'HAND'), ('GOOD', 'HEAD'), ('GOOD', 'HEART'), ('GOOD', 'HEAVY'), ('GOOD', 'ABOVE'), ('GOOD', 'HOT'), ('GREEN', 'HAMMER'), ('GREEN', 'HAND'), ('GREEN', 'HEAD'), ('GREEN', 'HEART'), ('GREEN', 'HEAVY'), ('GREEN', 'ABOVE'), ('GREEN', 'HOT'), ('HAMMER', 'HAND'), ('HAMMER', 'HEAD'), ('HAMMER', 'HEART'), ('HAMMER', 'HEAVY'), ('HAMMER', 'ABOVE'), ('HAMMER', 'HOT'), ('HAND', 'HEAD'), ('HAND', 'HEART'), ('HAND', 'HEAVY'), ('HAND', 'ABOVE'), ('HAND', 'HOT'), ('HEAD', 'HEART'), ('HEAD', 'HEAVY'), ('HEAD', 'ABOVE'), ('HEAD', 'HOT'), ('HEART', 'HEAVY'), ('HEART', 'ABOVE'), ('HEART', 'HOT'), ('HEAVY', 'ABOVE'), ('HEAVY', 'HOT'), ('ABOVE', 'HOT')]
45


In [32]:
# check if total number is correct
# n! / (r! (n -- r)!)
import math
n = len(df_concept_small['Concepticon_Gloss'])
r = 2
tot = math.factorial(n)/(math.factorial(r) * math.factorial(n -r))
print(tot)

45.0


In [33]:
# create df with combinations
df_comb=pd.DataFrame()
df_comb[['Concepticon_Gloss_1', 'Concepticon_Gloss_2']]= pd.DataFrame(comb)

In [34]:
# add ID 1 column
df_comb=df_comb.join ( df_concept_small.set_index( [ 'Concepticon_Gloss' ], verify_integrity=True ),
               on=[ 'Concepticon_Gloss_1' ], how='left' )
# rename ID 1 column
df_comb.rename(columns={'Concepticon_ID':'Concepticon_ID_1'}, inplace=True)

# add ID 2 column
df_comb=df_comb.join ( df_concept_small.set_index( [ 'Concepticon_Gloss' ], verify_integrity=True ),
               on=[ 'Concepticon_Gloss_2' ], how='left' )
# rename ID 2 column
df_comb.rename(columns={'Concepticon_ID':'Concepticon_ID_2'}, inplace=True)

In [35]:
df_comb.head()

,Concepticon_Gloss_1,Concepticon_Gloss_2,Concepticon_ID_1,Concepticon_ID_2
0,GOLD,GOOD,1369,1035
1,GOLD,GREEN,1369,1425
2,GOLD,HAMMER,1369,1481
3,GOLD,HAND,1369,1277
4,GOLD,HEAD,1369,1256
